In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_03 import *

In [3]:
path= Path('../PCImages')
path2fm= '../FeatureMatsMerged/TheGreatCollection.txt'
path2colnames= 'FeatureMatIndex.txt'

In [4]:
fm= fm_from_txt(path2fm, path2colnames)
fm.shape

(56657, 105)

In [5]:
cols_filtered=get_filtered_cols(fm)
len(cols_filtered)

44

## Mixup dataset

In [6]:
#export
class NumpyToFloatTensor(object): 
    def __call__(self, img): return torch.from_numpy(img)

## Create a new object data_files with 

Object `load_file` not found.


In [103]:
class CellDataSet(Dataset):
    def __init__(self, path, fm, cols2keep, label = 'label1', mixup_cs=[.4, .3, .3], trfms=None, random_sample=None):
        self.fm = fm
        self.cols2keep= cols2keep
        self.y = self.fm[label]
        self.id2label= id2label(self.fm.id, self.y)
        self.data_files = get_filtered_files(path, img_ids=self.fm.id, labels=self.y) 
        if random_sample is not None: self.fm= self.fm.sample(n=random_sample, random_state=0)
        self.trfms = get_trfms(trfms)
        self.cs= mixup_cs
        

    def __len__(self):
        return len(self.fm.id)
    
    def get_row(self, img_id):
        x_np= np.array(self.fm.loc[self.fm['id'].isin([img_id])][self.cols2keep]).flatten() 
        return torch.cuda.FloatTensor(x_np)
    
    def __getitem__(self, idx):
        print(idx)
        img_id  = self.fm.id[idx]
        label=          self.id2label[img_id]
        img_names= [f for f in self.data_files if img_id in split_fn(f)[0]]
        img1, img2, img3 = [PIL.Image.open(name) for name in img_names]
        image= self.cs[0]*np.array(img1) + self.cs[1]*np.array(img2) + self.cs[2]*np.array(img3)
        fm_row=         self.get_row(img_id)
        
        if self.trfms:
            image = self.trfms(image)

        return image, fm_row, label

In [104]:
ds= CellDataSet(path, fm, cols_filtered, label = 'label1', 
                mixup_cs=[.4, .3, .3],
                trfms= [
                    #transforms.Grayscale(num_output_channels=1), 
                    NumpyToFloatTensor()], 
                random_sample=1000)

In [105]:
len(ds), fm.shape

(1000, (56657, 105))

In [106]:
learn= Learner(ds, CombinedModel(50,10, p=0.5, n_features=44))

0


KeyError: 0

In [ ]:
learn.fit(1, bs=8, lr=1e-5)

In [108]:
fm.id[1]

'180322140233'

In [23]:
data_files = get_filtered_files(path, img_ids=fm.id, labels=fm.label1) 

In [24]:
data_files[:5]

[PosixPath('../PCImages/171220_170148_stretch.png'),
 PosixPath('../PCImages/161125_131633_relax.png'),
 PosixPath('../PCImages/161125_131633_stretch.png'),
 PosixPath('../PCImages/161125_131633_trap.png'),
 PosixPath('../PCImages/161125_131649_relax.png')]

In [27]:
len(data_files)

145775

In [25]:
img_id = '171220_170148'

In [34]:
img_name  = data_files[0]
img_id, phase = split_fn(img_name)

In [35]:
img_id

'171220170148'

In [43]:
split_fn(img_name)

('171220170148', 'stretch')

In [44]:
imgs= [fn for fn in data_files if img_id in split_fn(fn)[0]]
imgs

[PosixPath('../PCImages/171220_170148_stretch.png'),
 PosixPath('../PCImages/171220_170148_relax.png'),
 PosixPath('../PCImages/171220_170148_trap.png')]

In [48]:
im1, im2, im3= [PIL.Image.open(name) for name in imgs]

In [ ]:
t= [.4, .3, .3]
ims= t[0]*np.array(im1)+t[1]*np.array(im2)+t[2]*np.array(im3)

In [ ]:
plt.imshow(im1, cmap='gray');

In [ ]:
plt.imshow(im2, cmap='gray');

In [ ]:
plt.imshow(im3, cmap='gray');

In [ ]:
plt.imshow(ims, cmap='gray');

In [ ]:
ToFloatTensor??